Nota generada a partir de [liga](https://www.dropbox.com/s/yjijtfuky3s5dfz/2.5.Compute_Unified_Device_Architecture.pdf?dl=0)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --gpus all --rm -v $(pwd):/datos --name jupyterlab_nvidia_cuda_c_container -p 8888:8888 -d palmoreck/jupyterlab_nvidia_cuda_c:1.1.0_10.2
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_nvidia_cuda_c_container
```

Documentación de la imagen de docker `palmoreck/jupyterlab_nvidia_cuda_c:1.1.0_10.2` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/nvidia/cuda_c).

---

**Nota: si se desean ejecutar los ejemplos que se presentan a continuación, es necesario tener una tarjeta gráfica NVIDIA.**

# CUDA C y generalidades de CUDA y GPU

## CUDA C

Consiste en extensiones al lenguaje C y en una *runtime library*. Ver [2.3.CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA.ipynb) para más información.

### Kernel

* En CUDA C se define una función que se ejecuta en el device y que se le nombra **kernel**. El *kernel* inicia con la sintaxis:

```
__global__ void mifun(int param){
...
}

```

y siempre es tipo `void` (no hay `return`).

* El llamado al *kernel* se realiza desde el host y con una sintaxis en la que se define el número de threads y bloques que serán utilizados para la ejecución del kernel. La sintaxis que se utiliza es con `<<< >>>` y en la primera entrada se coloca el número de bloques y en la segunda entrada el número de *threads*:


```
__global__ void mifun(int param){
...
}

int main(){
    int par;
    mifun<<<N,5>>> (par); //N bloques de 5 threads
}

```

## Ejemplos

### 1) Programa de hello world

In [1]:
%%file hello_world.cu
#include<stdio.h>
__global__ void func(void){
}
int main(void){
    func<<<1,1>>>(); //1 bloque de 1 thread
    printf("Hello world!\n");
return 0;
}

Writing hello_world.cu


Compilación:

In [11]:
%%bash
nvcc --compiler-options -Wall hello_world.cu -o hello_world.out

Ejecución:

In [4]:
%%bash
./hello_world.out

Hello world!


**Comentarios:**

* La función `main` se ejecuta en la CPU.

* La función `func` es un *kernel* y es ejecutada por *threads* en el *device* (GPU), también llamados **CUDA threads**. Obsérvese que la función `func` inicia con `__global__` para diferenciarla de `main`. En este caso el *thread* que fue lanzado no realiza ninguna acción pues el cuerpo del kernel está vacío.

* El *kernel* sólo puede tener un `return` tipo *void*: `__global__ void func` por lo que el *kernel* debe regresar sus resultados a través de sus argumentos.
 

### 2) Programa de hello world 2

In [5]:
%%file hello_world_2.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<2,3>>>(); //2 bloques de 3 threads cada uno
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}


Writing hello_world_2.cu


In [12]:
%%bash 
nvcc --compiler-options -Wall hello_world_2.cu -o hello_world_2.out

In [7]:
%%bash
./hello_world_2.out

Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hola del cpu thread


**Comentarios:**

* La extensión del archivo debe ser `.cu` aunque esto puede modificarse al compilar con `nvcc`: 

`$nvcc -x cu hello_world.c -o hello_world.out`

* El llamado a la ejecución del kernel se realizó en el *host* y se lanzaron $2$ bloques (primera posición), cada uno con $3$ *threads*.

* Se utiliza la función [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) para que el *cpu-thread* espere la finalización de la ejecución del kernel.

* Los *CUDA threads* son divididos en bloques, **CUDA blocks** y todos los bloques se encuentran en un **grid**. En el lanzamiento del *kernel*  se debe especificar al hardware cuántos bloques tendrá nuestro *grid* y cuántos *threads* estarán en cada bloque. Las variables `blockIdx` y `threadIdx` hacen referencia a los **id**'s que tienen los bloques y los threads. El *id* del bloque dentro del *grid* y el *id* del thread dentro del bloque. La parte `.x` de `blockIdx.x` y `threadIdx.x` refiere a la **primera coordenada** del bloque en el *grid* y del *thread* en en el bloque. 

* La elección del número de bloques en un grid o el número de *threads* en un bloque no corresponde a alguna disposición del hardware, esto es, si se lanza un kernel con `<<< 1, 3 >>>` no implica que la GPU tenga en su hardware un bloque o 3 *threads*.

* En una GPU podemos definir el *grid* de bloques y el bloque de *threads* utilizando el tipo de dato `dim3` el cual también es parte de CUDA C:

In [14]:
%%file hello_world_3.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    dim3 dimGrid(2,1,1); //2 bloques en el grid
    dim3 dimBlock(3,1,1); //3 threads por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}

Overwriting hello_world_3.cu


In [15]:
%%bash 
nvcc --compiler-options -Wall hello_world_3.cu -o hello_world_3.out

In [16]:
%%bash
./hello_world_3.out

Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hola del cpu thread


**Obs:** obsérvese que puede definirse un grid de tres dimensiones y también un bloque de tres dimensiones.

### 3) Programa de suma vectorial

**N bloques de 1 thread**

In [1]:
%%file suma_vectorial.cu
#include<stdio.h>
#define N 10
__global__ void suma_vect(int *a, int *b, int *c){
    int block_id_x = blockIdx.x;
    if(block_id_x<N) //aquí se asume que el valor de N 
                     //es menor al número máximo de bloques que se pueden lanzar
                     //si fuese mayor, hay que hacer un ajuste
        c[block_id_x] = a[block_id_x]+b[block_id_x];
}
int main(void){
    int a[N], b[N],c[N];
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(N,1,1); //N bloques en el grid
    dim3 dimBlock(1,1,1); //1 threads por bloque 
    //alojando en device
    cudaMalloc((void **)&device_a, sizeof(int)*N); 
    cudaMalloc((void **)&device_b, sizeof(int)*N);
    cudaMalloc((void **)&device_c, sizeof(int)*N);
    //llenando los arreglos con datos dummy:
    for(i=0;i<N;i++){
        a[i]=i;
        b[i]=i*i;
    }
    //copiamos arreglos a, b a la GPU
    cudaMemcpy(device_a,a,N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(device_b,b,N*sizeof(int), cudaMemcpyHostToDevice);
    //mandamos a llamar a suma_vect:
    suma_vect<<<dimGrid,dimBlock>>>(device_a,device_b,device_c); //N bloques de 1 thread
    cudaDeviceSynchronize();
    //copia del resultado al arreglo c:
    cudaMemcpy(c,device_c,N*sizeof(int),cudaMemcpyDeviceToHost);
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",a[i],b[i],c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Overwriting suma_vectorial.cu


In [2]:
%%bash
nvcc --compiler-options -Wall suma_vectorial.cu -o suma_vectorial.out

In [3]:
%%bash
./suma_vectorial.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


**Comentarios:**

* Obsérvese que se están utilizando apuntadores en la línea:

```
    int *device_a, *device_b, *device_c;
```

pero estos apuntadores no apuntan a una dirección de memoria en el *device* pues aunque NVIDIA añadió el *feature* de [Unified Memory](https://devblogs.nvidia.com/unified-memory-cuda-beginners/) (un espacio de memoria accesible para la CPU y la GPU) aquí no se está usando tal *feature*. Más bien se están utilizando los apuntadores anteriores para apuntar a un [struct](https://en.wikipedia.org/wiki/Struct_(C_programming_language)) de C en el que uno de sus tipos de datos es una dirección de memoria en la GPU.

* Para alojar memoria en la GPU se utiliza el llamado a [cudaMalloc](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1g37d37965bfb4803b6d4e59ff26856356) y para transferir datos del *host* al *device* o viceversa se llama a lafunción [cudaMemcpy](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gc263dbe6574220cc776b45438fc351e8) con respectivos parámetros como `cudaMemcpyHostToDevice` o `cudaMemcpyDeviceToHost`. Obsérvese el uso de `(void **)` por la definición de la función `cudaMalloc`.

* Para desalojar memoria en la GPU se utiliza el llamado a [cudaFree](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ga042655cbbf3408f01061652a075e094)

Al instalar el *CUDA toolkit* o con el contenedor de docker (detallado al inicio de la nota) se cuenta con la línea de comando [nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html) para perfilamiento (aunque en la documentación se menciona que será reemplazada tal línea de comando por [NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute) y [NVIDIA Nsight Systems](https://developer.nvidia.com/nsight-systems))

In [4]:
%%bash 

nvprof --normalized-time-unit s ./suma_vectorial.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==78== NVPROF is profiling process 78, command: ./suma_vectorial.out
==78== Profiling application: ./suma_vectorial.out
==78== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    37.80  1.54e-06         1  1.54e-06  1.54e-06  1.54e-06  suma_vect(int*, int*, int*)
                    34.65  1.41e-06         2  7.04e-07  5.12e-07  8.96e-07  [CUDA memcpy HtoD]
                    27.56  1.12e-06         1  1.12e-06  1.12e-06  1.12e-06  [CUDA memcpy DtoH]
      API calls:    99.47  0.094267         3  0.031422  4.82e-06  0.094256  cudaMalloc
                     0.31  2.92e-04        97  3.01e-06  3.04e-07  1.59e-04  cuDeviceGetAttribute
                     0.08  7.31e-05         1  7.31e-05  7.31e-05  7.31e-05  cuDeviceTotalMem
                     0.06  5.44e-05         3  1.81e-05  3.31e-06  4.51e-05  cudaFree
                     0.03  2.75e-0

Unidades en las que se reporta, s: second, ms: millisecond, us: microsecond, ns: nanosecond

**1 bloque de N threads**

Y en lugar de lanzar $N$ bloques de $1$ thread se puede lanzar $1$ bloque con $N$ threads:

In [7]:
%%file suma_vectorial_2.cu
#include<stdio.h>
#define N 10
__global__ void suma_vect(int *a, int *b, int *c){
    int thread_id_x = threadIdx.x;
    if(thread_id_x<N) //aquí se asume que el valor de N 
                     //es menor al número máximo de threads que se pueden lanzar
                    //si fuese mayor, hay que hacer un ajuste
        c[thread_id_x] = a[thread_id_x]+b[thread_id_x];
}
int main(void){
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(1,1,1); //1 bloques en el grid
    dim3 dimBlock(N,1,1); //N threads por bloque 
    //alojando en device con Unified Memory
    cudaMallocManaged(&device_a, sizeof(int)*N);
    cudaMallocManaged(&device_b, sizeof(int)*N);
    cudaMallocManaged(&device_c, sizeof(int)*N);
    //llenando los arreglos:
    for(i=0;i<N;i++){
        device_a[i]=i;
        device_b[i]=i*i;
    }
    suma_vect<<<dimGrid,dimBlock>>>(device_a,device_b,device_c); //1 bloque con N threads
    cudaDeviceSynchronize();
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",device_a[i],device_b[i],device_c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Overwriting suma_vectorial_2.cu


In [8]:
%%bash
nvcc --compiler-options -Wall suma_vectorial_2.cu -o suma_vectorial_2.out

In [9]:
%%bash
./suma_vectorial_2.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


In [10]:
%%bash 

nvprof --normalized-time-unit s ./suma_vectorial_2.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==164== NVPROF is profiling process 164, command: ./suma_vectorial_2.out
==164== Profiling application: ./suma_vectorial_2.out
==164== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:   100.00  2.05e-06         1  2.05e-06  2.05e-06  2.05e-06  suma_vect(int*, int*, int*)
      API calls:    99.33  0.106555         3  0.035518  1.20e-05  0.106522  cudaMallocManaged
                     0.31  3.32e-04         1  3.32e-04  3.32e-04  3.32e-04  cudaLaunchKernel
                     0.16  1.70e-04        97  1.75e-06  2.92e-07  6.53e-05  cuDeviceGetAttribute
                     0.09  9.75e-05         3  3.25e-05  1.20e-05  6.10e-05  cudaFree
                     0.07  7.17e-05         1  7.17e-05  7.17e-05  7.17e-05  cuDeviceTotalMem
                     0.03  2.68e-05         1  2.68e-05  2.68e-05  2.68e-05  cuDeviceGetName
                     0.0

**Obs:** obsérvese que el programa anterior utiliza la *Unified Memory* con la función [cudaMallocManaged](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__HIGHLEVEL.html#group__CUDART__HIGHLEVEL_1gcf6b9b1019e73c5bc2b39b39fe90816e).

El siguiente programa basado en [liga](https://devblogs.nvidia.com/how-query-device-properties-and-handle-errors-cuda-cc/) y [cudaDeviceProp Struct Reference](https://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html)

In [63]:
%%file device_properties.cu

#include<stdio.h>

int main(void){
    cudaDeviceProp properties;
    int count;
    int i;
    cudaGetDeviceCount(&count);
    for(i=0;i<count;i++){
        printf("----------------------\n");
        cudaGetDeviceProperties(&properties, i);
        printf("----device %d ----\n",i); 
        printf("Device Name: %s\n", properties.name);
        printf("Compute capability: %d.%d\n", properties.major, properties.minor);
        printf("Clock rate: %d\n", properties.clockRate);
        printf("Unified memory: %d\n", properties.unifiedAddressing);
        printf(" ---Memory Information for device %d (results on bytes)---\n", i);
        printf("Total global mem: %ld\n", properties.totalGlobalMem); 
        printf("Total constant Mem: %ld\n", properties.totalConstMem);
        printf("Shared memory per thread block: %ld\n", properties.sharedMemPerBlock);
        printf("Shared memory per SM: %ld\n",properties.sharedMemPerMultiprocessor );
        printf(" ---MP Information for device %d ---\n", i);
        printf("SM count: %d\n", properties.multiProcessorCount);
        printf("Threads in warp: %d\n", properties.warpSize);
        printf("Max threads per SM: %d\n", properties.maxThreadsPerMultiProcessor);
        printf("Max warps per SM: %d\n",properties.maxThreadsPerMultiProcessor/properties.warpSize);
        printf("Max threads per block: %d\n", properties.maxThreadsPerBlock);
        printf("Max thread dimensions: (%d, %d, %d)\n", properties.maxThreadsDim[0], properties.maxThreadsDim[1], properties.maxThreadsDim[2]);
        printf("Max grid dimensions: (%d, %d, %d)\n", properties.maxGridSize[0], properties.maxGridSize[1], properties.maxGridSize[2]); 
    }
    return 0;
    
}

Overwriting device_properties.cu


In [64]:
%%bash

nvcc --compiler-options -Wall device_properties.cu -o device_properties.out

In [66]:
%%bash

./device_properties.out

----------------------
----device 0 ----
Device Name: GeForce GTX 750
Compute capability: 5.0
Clock rate: 1293500
Unified memory: 1
 ---Memory Information for device 0 (results on bytes)---
Total global mem: 1025769472
Total constant Mem: 65536
Shared memory per thread block: 49152
Shared memory per SM: 65536
 ---MP Information for device 0 ---
SM count: 4
Threads in warp: 32
Max threads per SM: 2048
Max warps per SM: 64
Max threads per block: 1024
Max thread dimensions: (1024, 1024, 64)
Max grid dimensions: (2147483647, 65535, 65535)


### Regla compuesta del rectángulo

In [36]:
%%file Rcf.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum ) {
double x=0.0;

if(threadIdx.x<=n-1){
x=a+(threadIdx.x+1/2.0)*h_hat;
data[threadIdx.x]=std::exp(-std::pow(x,2));
}
    *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
}

int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1e3; //número de subintervalos
    double objetivo=0.7468241328124271;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_suma); //1 bloque de n threads
    cudaDeviceSynchronize();
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    return 0;
}

Writing Rcf.cu


In [37]:
%%bash
nvcc --compiler-options -Wall Rcf.cu -o Rcf.out

In [38]:
%%bash
./Rcf.out

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01
Error relativo de la solución: 4.104931878976858e-08


In [40]:
%%bash
nvprof --normalized-time-unit s ./Rcf.out

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01
Error relativo de la solución: 4.104931878976858e-08


==666== NVPROF is profiling process 666, command: ./Rcf.out
==666== Profiling application: ./Rcf.out
==666== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    99.41  2.33e-04         1  2.33e-04  2.33e-04  2.33e-04  Rcf(double*, double, double, int, double*)
                     0.59  1.38e-06         1  1.38e-06  1.38e-06  1.38e-06  [CUDA memcpy DtoH]
      API calls:    99.19  0.093274         2  0.046637  5.95e-06  0.093269  cudaMalloc
                     0.33  3.13e-04        97  3.23e-06  3.34e-07  1.67e-04  cuDeviceGetAttribute
                     0.25  2.36e-04         1  2.36e-04  2.36e-04  2.36e-04  cudaDeviceSynchronize
                     0.09  8.47e-05         1  8.47e-05  8.47e-05  8.47e-05  cuDeviceTotalMem
                     0.05  5.16e-05         2  2.58e-05  6.01e-06  4.56e-05  cudaFree
                     0.03  3.13e-0

**Referencias**

1. N. Matloff, Parallel Computing for Data Science. With Examples in R, C++ and CUDA, 2014.

2. [CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA)

3. [2.3.CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA.ipynb)

Para más sobre *Unified Memory* revisar:

* [Even easier introduction to cuda](https://devblogs.nvidia.com/even-easier-introduction-cuda/)

* [Unified memory cuda beginners](https://devblogs.nvidia.com/unified-memory-cuda-beginners/)